# Introduction

In this guide, we will develop an intuition for the basic principles and concepts of web development. We will do so using the FastHTML library, which enables us to create websites and web applications using Python. Crucially, it still requires considerable understanding of how web pages and web apps are built and how they work. It won't make it easy to build web pages without understanding HTML/CSS or HTTP requests. But it might mean that there is now a "Pythonic" way to learn it and to get started quickly.

The goal of this guide is to start as simply as possible and to add more and more complexity to that simple start, explaining what is happening at each step and mapping the FastHTML code to a broader understanding of web development concepts.

## A Simple Start
As always...let's start with a Hello, World. In this case, we will develop a web page that says "Hello, World" that you can serve and see in your browser.

In [2]:
from fasthtml.common import *

app, rt = fast_app()

@rt("/")
def get():
    return Titled("FastHTML", P("Hello, World!"))

serve()

To see the results, save this as a `.py` file and run it with `python <filename>.py`. There are a few things happening here that are immediately worth pointing out:
1. The `@rt("/")` decorator tells us the *route* with which the decorated function will be associated. `/` is just the index page or main page.
2. `get()` isn't just an arbitrary function name—it is an HTTP verb or [HTTP request method](hhttps://developer.mozilla.org/en-US/docs/Web/HTTP/Methods). HTTP methods specify the type of action to be performed; `get` means that we want to retrieve some data. Note that FastHTML gives a few different ways of specifying routes and methods. We could have decorated a differently-named function with `@app.get("/")`. The following cell does the same thing; replace `app.py` with its contents and run it again to see.

In [6]:
%%writefile app_0.py
from fasthtml.common import *

app, rt = fast_app()

@app.get("/")
def hello():
    return Titled("Hello, World!")

serve()

Overwriting app_0.py


To recap what happens: when a user goes to the home page (`/`), a `get()` http request is triggered, which, in this case, calls the `hello()` function, which generates the resulting "Hello, World" page.

But what does "generates the resulting "Hello, World" page actually mean? 